In [1]:
from instance_runner import InstanceRunner
import pandas as pd
import json
from tqdm import tqdm
import os

In [2]:
runs = 1
output_folder = "/home/pedrowsl/cbm/output/01_09/"
output_file = f"{output_folder}results_01_09.csv"
instances = ["a1", "a2", "a3", "a4", "a5", "b1", "b2", "b3", "b4", "b5", "c1", "c2", "c3", "c4", "c5", "d1", "d2", "d3", "d4", "d5", "e1", "e2", "e3", "e4", "e5", "f1", "f2", "f3", "f4", "f5", "g1", "g2", "g3", "g4", "g5", "h1", "h2", "h3", "h4", "h5", "i1", "i2", "i3", "i4", "i5"]
soares = [215, 218, 208, 237, 231, 641, 637, 628, 648, 621, 1259, 1259, 1262, 1270, 1328, 495, 505, 446, 477, 459, 1445, 1434, 1420, 1417, 1465, 3006, 3052, 3051, 3061, 3078, 904, 896, 923, 945, 874, 2820, 2715, 2807, 2636, 2898, 5967, 6001, 5888, 5920, 5920]
haddadi = [210, 216, 204, 233, 226, 633, 631, 620, 638, 613, 1248, 1245, 1249, 1256, 1312, 491, 502, 445, 472, 454, 1420, 1406, 1399, 1391, 1440, 2959, 3010, 3006, 3017, 3032, 893, 884, 912, 932, 863, 2721, 2614, 2707, 2538, 2793, 5751, 5795, 5681, 5704, 5708]
exact = [210, 216, 204, 233, 226, 632, 629, 619, 638, 613, 1245, 1242, 1247, 1253, 1311, 491, 502, 445, 472, 454, 1412, 1397, 1394, 1386, 1430, 2955, 3002, 2999, 3009, 3021, 893, 884, 912, 932, 763, 2708, 2597, 2688, 2516, 2780, 5714, 5767, 5653, 5674, 5676]
parameters = {
    "threads": 15,
    "tempL": 15,
    "lkhS": 5,
    "tempMin": 0.01,
    "tempMax": 2.25,
    "constructionBias": 5.0,
    "MKL": 1000,
    "PTL": 20000,
    "tempD": 3,
    "upType": 1,
    "tempUpdate": 7,
    "movementType": 2,
}

In [3]:
file_exists = os.path.isfile(output_file)

for instance, s, h, e in tqdm(zip(instances, soares, haddadi, exact)):
    for _ in range(runs):
        ir = InstanceRunner(
            parameters=parameters,
            root_path="/home/pedrowsl/cbm",
            instance_path="/home/pedrowsl/cbm/instances/" + instance,
            header_path="/home/pedrowsl/cbm/src/CBMProblem.hpp",
        )
        output, elapsed = ir.run()
        output = json.loads(output)

        df_row = pd.DataFrame([{
            'instance': instance,
            'exact': e,
            'pt': output['final_solution']['cost'],
            'lkh': min(x['cost'] for x in output['initial_solutions'] if x['construction'] == 'LKH'),
            'haddadi': h,
            'soares': s,
            'solution': output['final_solution']['solution'],
            't(s)': elapsed
        }])

        with open(f"{output_folder}{instance}.json", "w") as f:
            f.write(json.dumps(output, indent=4)) # pretty printing
        df_row.to_csv(output_file, mode='a', header=not file_exists, index=False)
        file_exists = True

15it [03:30, 13.45s/it]

g++ -fopenmp -std=c++17 -O3 -march=native -c src/CBMSol.cpp -o src/CBMSol.o.prd
g++ -fopenmp -std=c++17 -O3 -march=native -c src/CBMProblem.cpp -o src/CBMProblem.o.prd


src/CBMProblem.cpp: In lambda function:
src/CBMProblem.cpp:129:5: warning: control reaches end of non-void function [-Wreturn-type]
  129 |     };
      |     ^


g++ -fopenmp -std=c++17 -O3 -march=native -c src/main.cpp -o src/main.o.prd
g++ -fopenmp -std=c++17 -O3 -march=native -o main_prd src/CBMSol.o.prd src/CBMProblem.o.prd src/main.o.prd
Compilation finished successfully.


30it [10:19, 21.83s/it]

g++ -fopenmp -std=c++17 -O3 -march=native -c src/CBMSol.cpp -o src/CBMSol.o.prd
g++ -fopenmp -std=c++17 -O3 -march=native -c src/CBMProblem.cpp -o src/CBMProblem.o.prd


src/CBMProblem.cpp: In lambda function:
src/CBMProblem.cpp:129:5: warning: control reaches end of non-void function [-Wreturn-type]
  129 |     };
      |     ^


g++ -fopenmp -std=c++17 -O3 -march=native -c src/main.cpp -o src/main.o.prd
g++ -fopenmp -std=c++17 -O3 -march=native -o main_prd src/CBMSol.o.prd src/CBMProblem.o.prd src/main.o.prd
Compilation finished successfully.


45it [40:31, 54.03s/it] 
